In [ ]:
# Simulation Data
names = ['foo','bar','rf']
dates = pd.date_range(start='2017-01-01',end='2017-12-31', freq=pd.tseries.offsets.BDay()) # Exclude Businness Day
n = len(dates)
rdf = pd.DataFrame(
    np.zeros((n, len(names))),
    index = dates,
    columns = names
)

np.random.seed(1)
rdf['foo'] = np.random.normal(loc = 0.1/n,scale=0.2/np.sqrt(n),size=n)
rdf['bar'] = np.random.normal(loc = 0.04/n,scale=0.05/np.sqrt(n),size=n)
rdf['rf'] = np.random.normal(loc=0.02/ n, scale=0.01 / np.sqrt(n), size=n)

pdf = 100*np.cumprod(1+rdf)
    # algo to fire on the beginning of every month and to run on the first date
pdf

In [ ]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        self.dateindex = self.datas[0].datetime
        self.sma = btind.SimpleMovingAverage(period=15)

    

    def next(self):
            if self.sma > self.data.close:
                print(self.sma,+' and ' ,self.dataclose)

        # Simply log the closing price of the series from the reference
#         print('#'*50)
#         print(f'ETH Close {self.datetime.datetime(0)} , ',self.dataclose[0])
  
#         self.log('Close, %.2f' % self.dataclose[0])

 class Test(bt.Strategy):
        params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )
        
        def __init__(self):
            sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
            sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
            self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

        def next(self):
            if not self.position:  # not in the market
                if self.crossover > 0:  # if fast crosses slow to the upside
                    self.buy()  # enter long

            elif self.crossover < 0:  # in the market & cross to the downside
                self.close()  # close long position

In [ ]:
from datetime import datetime
import backtrader as bt
import pandas as pd
from pandas_datareader import data as pdr

data = pdr.get_data_yahoo('TSLA', start=datetime(2017, 8, 13), end=datetime(2018, 8, 14))
data.columns=['high', 'low', 'open', 'close', 'volume', 'adj_close']


data['pct']=data.close.pct_change(1)
data['pct2']=data.close.pct_change(5)
data['pct3']=data.close.pct_change(10)




cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)

cerebro.addstrategy(Test)


class PandasData(bt.feeds.PandasData):
    lines = ('adj_close','pct','pct2','pct3')
    params = (
        ('datetime', None),
        ('open','open'),
        ('high','high'),
        ('low','low'),
        ('close','close'),
        ('volume','volume'),
        ('openinterest',None),
        ('adj_close','adj_close'),
        ('pct','pct'),
        ('pct2','pct2'),
        ('pct3','pct3'),
    )
    
    
df = bt.feeds.PandasData(dataname=ohlc)
# df=PandasData(dataname=data)

cerebro.adddata(df)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False, stdstats=False)


In [ ]:
colmuns_ohlc = ('date','open','high','low','close','volume')
ohlc =pd.DataFrame(ex.fetch_ohlcv(symbols[0],'1d'),columns=colmuns_ohlc)
ohlc['date']=pd.to_datetime(ohlc['date'],unit='ms')
ohlc.set_index('date',inplace=True)

In [ ]:
import datetime
import backtrader as bt
# https://algotrading101.com/learn/backtrader-for-backtesting/
# https://community.backtrader.com/topic/3676/how-to-backtest-using-portfolio-compositions/2
class Strategy(bt.Strategy):

    params = (
        ("buffer", 0.00),
        ("threshold", 0.01),
    )

    def log(self, txt, dt=None):
        """ Logging function fot this strategy"""
        dt = dt or self.data.datetime[0]
        if isinstance(dt, float):
            dt = bt.num2date(dt)
        print("%s, %s" % (dt.date(), txt))

    def print_signal(self):
        self.log(
            f"o {self.datas[0].open[0]:7.2f} "
            f"h {self.datas[0].high[0]:7.2f} "
            f"l {self.datas[0].low[0]:7.2f} "
            f"c {self.datas[0].close[0]:7.2f} "
            f"v {self.datas[0].volume[0]:7.0f} "
        )

    def notify_order(self, order):
        """ Triggered upon changes to orders. """
        # Suppress notification if it is just a submitted order.
        if order.status == order.Submitted:
            return

        # Print out the date, security name, order number and status.
        type = "Buy" if order.isbuy() else "Sell"
        self.log(
            f"{order.data._name:<6} Order: {order.ref:3d} "
            f"Type: {type:<5}\tStatus"
            f" {order.getstatusname():<8} \t"
            f"Size: {order.created.size:9.4f} Price: {order.created.price:9.4f} "
            f"Position: {self.getposition(order.data).size:5.2f}"
        )
        if order.status == order.Margin:
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            self.log(
                f"{order.data._name:<6} {('BUY' if order.isbuy() else 'SELL'):<5} "
                # f"EXECUTED for: {dn} "
                f"Price: {order.executed.price:6.2f} "
                f"Cost: {order.executed.value:6.2f} "
                f"Comm: {order.executed.comm:4.2f} "
                f"Size: {order.created.size:9.4f} "
            )

    def notify_trade(self, trade):
        """Provides notification of closed trades."""
        if trade.isclosed:
            self.log(
                "{} Closed: PnL Gross {}, Net {},".format(
                    trade.data._name,
                    round(trade.pnl, 2),
                    round(trade.pnlcomm, 1),
                )
            )

    def next(self):
        track_trades = dict()
        total_value = self.broker.get_value() #* (1 - self.p.buffer)

        for d in self.datas:
            track_trades[d] = dict()
            value = self.broker.get_value(datas=[d])
            allocation = value / total_value ### 0.0% - 1.0%
            units_to_trade = (d.target - allocation) * total_value / d.close[0] ## Sizing 
            track_trades[d]["units"] = units_to_trade

            # Can check to make sure there is enough distance away from ideal to trade.
            track_trades[d]["threshold"] = abs(d.target - allocation) > self.p.threshold ## ( fix [0.5]- now_value[0.45] > threshold) >
            ## abs(0.5- 0.55 ) > 0.02 (threshold)

        rebalance = False
        for values in track_trades.values():
            if values['threshold']:
                rebalance = True
#                 print(values)
                print(f'{allocation:.2f}%, Values {total_value:.2f}' )
 

        if not rebalance:
#             print(values)

            return
        # Sell shares first
        for d, value in track_trades.items():
            if value["units"] < 0:
                self.sell(d, size=value["units"])

        # Buy shares second
        for d, value in track_trades.items():
            if value["units"] > 0:
                self.buy(d, size=value["units"])


if __name__ == "__main__":

    cerebro = bt.Cerebro()

    tickers = {"FB": 0.50}#, "MSFT": 0.4, "TSLA": 0.35}

    for ticker, target in tickers.items():
        data = bt.feeds.YahooFinanceData(
            dataname=ticker,
            timeframe=bt.TimeFrame.Days,
            fromdate=datetime.datetime(2019, 1, 1),
            todate=datetime.datetime(2020, 12, 31),
            reverse=False,
        )
        data.target = target
        cerebro.adddata(data, name=ticker)

    cerebro.addstrategy(Strategy)

    # Execute
    cerebro.run()
#     cerebro.plot()